# Exercise on the value of unsupervised constructed features for training a classifier with few labeled examples

To get unsupervised constructed features of an image, we can use a pretrained CNN as feature extractor. 

We have done this to extract features from 100 Cifar10 images.  As pretrained CNN we use a VGG16 architecture that was trained on ImageNet data and was the second winner of the ImageNet competition in 2014. 

As a check on the quality of the feature representation of the CIFAR10 data, we will use once the pixel-features and once the VGG-features to train a classifier using this 100 labeled data (on average 10 per class). If the VGG-feature are indeed better than the raw pixel values, we would expect to achieve a better classifier when using the VGG-feature compared to the pixel feature.

a) Which accuracy would you expect for a classifier which cannot distinguish between the 10 classes and is only guessing?



b) Go through the code which is used to set-up, train, and evaluate a CNN classifier using the raw pixel features. Discuss your thoughts on the achieved accuracy (e.g. with your neighbor).



b) Now we use the unsupervised constructed VGG features. We want to check, if these VGG features are good enough to train a classifier with only few labeled data and still get a satisfying performance. For this purpose, please complete the code to set up a fully connected NN and run the provided subsequent code to train it and determine its accuracy on the test set. Compare it to the accuracy which we achieve with a RF. Discuss the results (e.g. with your neighbor).




## Imports

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from pylab import *

import time
import tensorflow as tf

import  tensorflow.keras as keras
import sys
print ("Keras {} TF {} Python {}".format(keras.__version__, tf.__version__, sys.version_info))

## CIFAR Data preparation

In [ ]:
#downlad cifar data
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
del [x_test,y_test]

In [ ]:
#loop over each class label and sample 100 random images over each label and save the idx to subset
np.random.seed(seed=222)
idx=np.empty(0,dtype="int8")
for i in range(0,len(np.unique(y_train))):
    idx=np.append(idx,np.random.choice(np.where((y_train[0:len(y_train)])==i)[0],100,replace=False))

x_train= x_train[idx]
y_train= y_train[idx]

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(np.unique(y_train,return_counts=True))

In [ ]:
#make train valid and test
#loop over each class label and sample 100 random images over each label and save the idx to subset
np.random.seed(seed=123)
idx_train=np.empty(0,dtype="int8")
for i in range(0,len(np.unique(y_train))):
    idx_train=np.append(idx_train,np.random.choice(np.where((y_train[0:len(y_train)])==i)[0],10,replace=False))

x_train_new = x_train[idx_train]
y_train_new = y_train[idx_train]

In [ ]:
x_test_new=(np.delete(x_train,idx_train,axis=0))
y_test_new=(np.delete(y_train,idx_train,axis=0))

In [ ]:
np.random.seed(seed=127)
idx_valid=np.empty(0,dtype="int8")
for i in range(0,len(np.unique(y_test_new))):
    idx_valid=np.append(idx_valid,np.random.choice(np.where((y_test_new[0:len(y_test_new)])==i)[0],10,replace=False))

x_valid_new = x_test_new[idx_valid]
y_valid_new = y_test_new[idx_valid]

In [ ]:
x_test_new=(np.delete(x_test_new,idx_valid,axis=0))
y_test_new=(np.delete(y_test_new,idx_valid,axis=0))

In [ ]:
x_train_new = np.reshape(x_train_new, (100,32,32,3))
x_valid_new = np.reshape(x_valid_new, (100,32,32,3))
x_test_new = np.reshape(x_test_new, (800,32,32,3))

In [ ]:
print(np.unique(y_train_new,return_counts=True))
print(np.unique(y_valid_new,return_counts=True))
print(np.unique(y_test_new,return_counts=True))

In [ ]:
from tensorflow.keras.utils import to_categorical   

y_train_new=to_categorical(y_train_new,10)
y_valid_new=to_categorical(y_valid_new,10)
y_test_new=to_categorical(y_test_new,10)



In [ ]:
print(x_train_new.shape)
print(y_train_new.shape)

print(x_valid_new.shape)
print(y_valid_new.shape)

print(x_test_new.shape)
print(y_test_new.shape)

In [ ]:
# center and standardize the data
X_mean = np.mean( x_train_new, axis = 0)
X_std = np.std( x_train_new, axis = 0)

x_train_new = (x_train_new - X_mean ) / (X_std + 0.0001)
x_valid_new = (x_valid_new - X_mean ) / (X_std + 0.0001)
x_test_new = (x_test_new - X_mean ) / (X_std + 0.0001)

## Baseline 1: use raw images to train a Random Forest model

In [ ]:
# reshape images for rf
x_train_rf = x_train_new.reshape(len(x_train_new),32*32*3)
x_valid_rf = x_valid_new.reshape(len(x_valid_new),32*32*3)
x_test_rf = x_test_new.reshape(len(x_test_new),32*32*3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(x_train_rf, np.argmax(y_train_new, axis=1))

In [ ]:
from sklearn.metrics import confusion_matrix
pred = clf.predict(x_test_rf)
print(confusion_matrix(np.argmax(y_test_new, axis=1), pred))
np.sum(pred == np.argmax(y_test_new, axis=1)) / len(np.argmax(y_test_new, axis=1))


## Setting up the the CNN classifier based on raw image data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten


In [ ]:
# here we define hyperparameter of the NN
batch_size = 10
nb_classes = 10
nb_epoch = 30
img_rows, img_cols = 32, 32
kernel_size = (3, 3)
input_shape = (img_rows, img_cols, 3)
pool_size = (2, 2)

In [ ]:
model = Sequential()

model.add(Convolution2D(8,kernel_size,padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution2D(8, kernel_size,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(16, kernel_size,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(16,kernel_size,padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(40))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history=model.fit(x_train_new, y_train_new, 
                  batch_size=10, 
                  epochs=30,
                  verbose=2, 
                  validation_data=(x_valid_new, y_valid_new),shuffle=True)

### Evaluation of the CNN classifier that was trained on raw image data

In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(x_test_new)
print(confusion_matrix(np.argmax(y_test_new,axis=1), np.argmax(pred,axis=1)))
print("Acc = " ,np.sum(np.argmax(y_test_new,axis=1)==np.argmax(pred,axis=1))/len(y_test_new))


## Getting the VGG features for CIFAR

In [ ]:
# Downloading embeddings
import urllib
import os
if not os.path.isfile('cifar_EMB_1000.npz'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/si287al91c1ls0d/cifar_EMB_1000.npz?dl=1",
    "cifar_EMB_1000.npz")
%ls -hl cifar_EMB_1000.npz

In [ ]:
Data=np.load("cifar_EMB_1000.npz")
vgg_features_cifar = Data["arr_0"]

In [ ]:
vgg_features_cifar_train = vgg_features_cifar[idx_train]
vgg_features_cifar_test=(np.delete(vgg_features_cifar,idx_train,axis=0))
vgg_features_cifar_valid = vgg_features_cifar_test[idx_valid]
vgg_features_cifar_test=(np.delete(vgg_features_cifar_test,idx_valid,axis=0))


In [ ]:
print(vgg_features_cifar_train.shape)
print(vgg_features_cifar_valid.shape)
print(vgg_features_cifar_test.shape)

## Baseline 2: use VGG feature to train a Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(vgg_features_cifar_train,np.argmax(y_train_new, axis=1))

In [ ]:
from sklearn.metrics import confusion_matrix
pred=clf.predict(vgg_features_cifar_test)
print(confusion_matrix(np.argmax(y_test_new, axis=1), pred))
np.sum(pred==np.argmax(y_test_new, axis=1))/len(np.argmax(y_test_new, axis=1))


## Setting up the the NN classifier based on VGG feature

In [ ]:
model = Sequential()
model.add(Dense(200,batch_input_shape=(None, 4096)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(200))

#### we still need to add the last layers to get the predictions on the 10 classes
####### your code here #######



####### end of your code #######


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history=model.fit(vgg_features_cifar_train, y_train_new, 
                  batch_size=10, 
                  epochs=20,
                  verbose=2, 
                  validation_data=(vgg_features_cifar_valid, y_valid_new),shuffle=True)

### Evaluation of the NN classifier that was trained on VGG features

In [ ]:
pred=model.predict(vgg_features_cifar_test)

#### we now want to get the confusion matrix for the predictions on the test data
########## your code here ##########



########## end of your code ##########
